# The flower girl problem

Assume that every morning you can buy some flower at cost $c$, during the day you can sell them at price $p$. At the end of the day you can keep the flowers for the next day.
During each day the demand is an integer uniformly taken between $1$ and $N$.

At the start of your first day you have no flower in stock. At the end of the $T^{th}$ day all stock is lost. You want to minimize your expected loss summed over the $T$ days. We will denote $u_t$ the number of flower bought at the beginning of day $t$. We assume that $u_t\in [0,M]$ (in particular we assume that it is continuous).

The parameters are set as follow.

In [1]:
using JuMP, Clp
 
c = 1
p = 2
N = 10
M = 7

7

## Question 1

If $T=1$, what is the flower girl problem ? 

The flower-girl for $T=1$ is the newsvendor problem.

In [2]:
T = 1

d = zeros(T,N)
for s in 1:N
    d[:,s] = s
end


m = Model(solver = ClpSolver())

@variable(m,0<= u1 <= M)
@variable(m,v1[1:N] >= 0)


for s in 1:N
    @constraint(m, v1[s] <= d[1,s]) # we sell no more than demand
    @constraint(m, v1[s] <= u1)    # we sell no more than stock
end


@objective(m, Min, c*u1 
                    + 1/N*sum( -p*v1[s] for s = 1:N) ) 

solve(m)
getobjectivevalue(m)

-3.0000000000000018

## Question 2

We consider now that $T=2$.

### Question 2.1 
From Question 1 deduce an upper bound to this problem.

Repeating the previous algorithm for the second will give us a solution ( where the stock is lost). Therefore we deduce that -6 is an upper bound of the problem for T=2.

### Question 2.2
Solve the flower-girl problem with a stochastic programming approach, by solving the associated extensive formulation.

In [3]:
T = 2

d = zeros(T,N)
for s in 1:N
    d[:,s] = s
end


m = Model(solver = ClpSolver())

@variable(m,0<= u1 <= M)
@variable(m,v1[1:N] >= 0)
@variable(m,0 <= u2[1:N] <= M)
@variable(m, v2[1:N,1:N] >= 0)

for s in 1:N
    @constraint(m, v1[s] <= d[1,s]) # we sell no more than demand
    @constraint(m, v1[s] <= u1)    # we sell no more than stock
    for ss in 1:N # demand on day 2
        @constraint(m, v2[s,ss] <= d[2,ss]) # we sell no more than demand
        @constraint(m, v1[s] + v2[s,ss] - u1 - u2[s]  <=  0)     # we sell no more than stock
    end
end


@objective(m, Min, c*u1 
                    + 1/N*sum( -p*v1[s] + c * u2[s] for s = 1:N) 
                    + 1/N^2 *  sum(-p *v2[s,ss] for s = 1:N, ss= 1:N )) 

solve(m)
getobjectivevalue(m)

-7.89999999999999

## Question 3

We assume here that $T=3$

### Question 3.1 

Solve the flower girl problem with a stochastic programming approach, by solving the associated extensive formulation. 


In [4]:
T = 3

d = zeros(T,N)
for s in 1:N
    d[:,s] = s
end


m = Model(solver = ClpSolver())

@variable(m,0<= u1 <= M)
@variable(m,v1[1:N] >= 0)
@variable(m,0 <= u2[1:N] <= M)
@variable(m, v2[1:N,1:N] >= 0)
@variable(m,0 <= u3[1:N, 1:N] <= M)
@variable(m, v3[1:N,1:N, 1:N] >= 0)

for s in 1:N
    @constraint(m, v1[s] <= d[1,s]) # we sell no more than demand
    @constraint(m, v1[s] <= u1)    # we sell no more than stock
    for ss in 1:N # demand on day 2
        @constraint(m, v2[s,ss] <= d[2,ss]) # we sell no more than demand
        @constraint(m, v1[s] + v2[s,ss] - u1 - u2[s]  <=  0)     # we sell no more than stock
        for sss in 1:N
            @constraint(m, v3[s,ss, sss] <= d[2,sss]) # we sell no more than demand
            @constraint(m, v1[s] + v2[s,ss] + v3[s, ss, sss] - u1 - u2[s] - u3[s, ss]  <=  0)     # we sell no more than stock
        end
    end
end


@objective(m, Min, c*u1 
                    + 1/N*sum( -p*v1[s] + c * u2[s] for s = 1:N) 
                    + 1/N^2 *  sum(-p *v2[s,ss] for s = 1:N, ss= 1:N )
                    + 1/N^2 *  sum( c * u3[s, ss] for s = 1:N, ss = 1:N) 
                    + 1/N^3 *  sum(-p *v3[s,ss, sss] for s = 1:N, ss= 1:N, sss= 1:N )) 

solve(m)
getobjectivevalue(m)

-13.037999999999878

## Question 3.2 

Write the flower-girl problem with splitted variables. Check that you obtain the same value as before.  

In [5]:
T = 3

d = zeros(T,N,N,N)
for s = 1:N, ss = 1:N, sss = 1:N
    d[1,s,ss,sss] = s
    d[2,s,ss,sss] = ss
    d[3,s,ss,sss] = sss
end


m = Model(solver = ClpSolver())

@variable(m,0<= u[1:T,1:N,1:N,1:N] <= M )
@variable(m, v[1:T,1:N,1:N,1:N] >= 0)

for t = 1:T, s = 1:N, ss = 1:N, sss = 1:N
    @constraint(m, v[t,s,ss,sss] <= d[t,s,ss,sss]) # we sell no more than demand
    @constraint(m, sum(v[tt,s,ss,sss]-u[tt,s,ss,sss] for tt=1:t) <= 0) # we sell no more than stock
end

    ### u1 is the same for all scenarios
for s = 1:N, ss = 1:N, sss = 1:N
    @constraint(m,u[1,s,ss,sss] == 1/N^3 *sum( u[1,t,tt,ttt] for t=1:N, tt=1:N, ttt=1:N))
end
        
        ### u2 and v1 are d1 measurable
for s = 1:N, ss = 1:N, sss = 1:N
    @constraint(m,u[2,s,ss,sss] == 1/N^2 *sum( u[2,s,tt,ttt] for tt=1:N, ttt=1:N))
    @constraint(m,v[1,s,ss,sss] == 1/N^2 *sum( v[1,s,tt,ttt] for tt=1:N, ttt=1:N))
end
    
        ### u3 and v2 are d1,d2 measurable
for s = 1:N, ss = 1:N, sss = 1:N
    @constraint(m,u[3,s,ss,sss] == 1/N *sum( u[3,s,ss,ttt] for ttt=1:N))
    @constraint(m,v[2,s,ss,sss] == 1/N *sum( v[2,s,ss,ttt] for ttt=1:N))
end

@objective(m, Min, 1/N^3*sum(c*u[t,s,ss,sss]-p*v[t,s,ss,sss] for t = 1:T, s = 1:N, ss = 1:N, sss = 1:N))

       
solve(m)
@show(getobjectivevalue(m))
                       

getobjectivevalue(m) = -13.037999999999819


-13.037999999999819

### Question 3.3

By suppressing constraints in the splitted version, find the lower-bound given by the anticipative approach.


In [6]:
                T = 3

d = zeros(T,N,N,N)
for s = 1:N, ss = 1:N, sss = 1:N
    d[1,s,ss,sss] = s
    d[2,s,ss,sss] = ss
    d[3,s,ss,sss] = sss
end


m = Model(solver = ClpSolver())

@variable(m,0<= u[1:T,1:N,1:N,1:N] <= M )
@variable(m, v[1:T,1:N,1:N,1:N] >= 0)

for t = 1:T, s = 1:N, ss = 1:N, sss = 1:N
    @constraint(m, v[t,s,ss,sss] <= d[t,s,ss,sss]) # we sell no more than demand
    @constraint(m, sum(v[tt,s,ss,sss]-u[tt,s,ss,sss] for tt=1:t) <= 0) # we sell no more than stock
end

@objective(m, Min, 1/N^3*sum(c*u[t,s,ss,sss]-p*v[t,s,ss,sss] for t = 1:T, s = 1:N, ss = 1:N, sss = 1:N))

       
solve(m)
@show(getobjectivevalue(m))
                              

getobjectivevalue(m) = -15.454999999999973


-15.454999999999973

### Question 3.4

By suppressing constraint in the splitted version, find the lower-bound given by the 2 stage-approach.

In [7]:
                T = 3

d = zeros(T,N,N,N)
for s = 1:N, ss = 1:N, sss = 1:N
    d[1,s,ss,sss] = s
    d[2,s,ss,sss] = ss
    d[3,s,ss,sss] = sss
end


m = Model(solver = ClpSolver())

@variable(m,0<= u[1:T,1:N,1:N,1:N] <= M )
@variable(m, v[1:T,1:N,1:N,1:N] >= 0)

for t = 1:T, s = 1:N, ss = 1:N, sss = 1:N
    @constraint(m, v[t,s,ss,sss] <= d[t,s,ss,sss]) # we sell no more than demand
    @constraint(m, sum(v[tt,s,ss,sss]-u[tt,s,ss,sss] for tt=1:t) <= 0) # we sell no more than stock
end

     ##u1 is the same for all scenarios
for s = 1:N, ss = 1:N, sss = 1:N
    @constraint(m,u[1,s,ss,sss] == 1/N^3 *sum( u[1,t,tt,ttt] for t=1:N, tt=1:N, ttt=1:N))
end

@objective(m, Min, 1/N^3*sum(c*u[t,s,ss,sss]-p*v[t,s,ss,sss] for t = 1:T, s = 1:N, ss = 1:N, sss = 1:N))

       
solve(m)
@show(getobjectivevalue(m))
                              

getobjectivevalue(m) = -15.419999999999918


-15.419999999999918

## Question 4

We assume now that we work on a week, that is $T=5$. 

### Question 4.1  
how many variables have the extended formulation ? 

As seen for T=3 , we have s, ss, sss all in $[1:N]$. U and V are of dimention NxNxN
this means for T=5 we will have s, ss, sss, ssss, sssss in the same space. 
Therefore we get $2 * N^{5}= 2 * 10^{5}$ variables. A lot

### Question 4.2 (Optional)

Estimate the lower-bound given by the anticipative approach.

In [8]:
T = 5

d = zeros(T,N,N,N, N, N)
for s = 1:N, ss = 1:N, sss = 1:N, ssss = 1:N, sssss = 1:N
    d[1,s,ss,sss,ssss,sssss] = s
    d[2,s,ss,sss,ssss,sssss] = ss
    d[3,s,ss,sss,ssss,sssss] = sss
    d[4,s,ss,sss,ssss,sssss] = ssss
    d[5,s,ss,sss,ssss,sssss] = sssss
end


m = Model(solver = ClpSolver())

@variable(m,0<= u[1:T,1:N,1:N,1:N,1:N,1:N] <= M )
@variable(m, v[1:T,1:N,1:N,1:N,1:N,1:N] >= 0)

for t = 1:T, s = 1:N, ss = 1:N, sss = 1:N, ssss = 1:N, sssss = 1:N
    @constraint(m, v[t,s,ss,sss,ssss,sssss] <= d[t,s,ss,sss,ssss,sssss]) # we sell no more than demand
    @constraint(m, sum(v[tt,s,ss,sss,ssss,sssss]-u[tt,s,ss,sss,ssss,sssss] for tt=1:t) <= 0) # we sell no more than stock
end

@objective(m, Min, 1/N^5*sum(c*u[t,s,ss,sss, ssss, sssss]-p*v[t,s,ss,sss, ssss, sssss] for t = 1:T, s = 1:N, ss = 1:N, sss = 1:N, ssss = 1:N, sssss = 1:N))

       
solve(m)
@show(getobjectivevalue(m))

getobjectivevalue(m) = -26.270979999921202


-26.270979999921202

### Question 4.3 (Optional)

Evaluate the heuristic consisting in buying the same amount $u$ of flower everyday, for $u = 1..m$, and give an upper bound to the problem.

In [9]:

for i in 1:10
    
    T = 5

    d = zeros(T,N,N,N, N, N)
    for s = 1:N, ss = 1:N, sss = 1:N, ssss = 1:N, sssss = 1:N
        d[1,s,ss,sss,ssss,sssss] = s
        d[2,s,ss,sss,ssss,sssss] = ss
        d[3,s,ss,sss,ssss,sssss] = sss
        d[4,s,ss,sss,ssss,sssss] = ssss
        d[5,s,ss,sss,ssss,sssss] = sssss
    end

    u = ones(T,N,N,N, N, N) * i

    m = Model(solver = ClpSolver())

    @variable(m, v[1:T,1:N,1:N,1:N,1:N,1:N] >= 0)

    for t = 1:T, s = 1:N, ss = 1:N, sss = 1:N, ssss = 1:N, sssss = 1:N
        @constraint(m, v[t,s,ss,sss,ssss,sssss] <= d[t,s,ss,sss,ssss,sssss]) # we sell no more than demand
        @constraint(m, sum(v[tt,s,ss,sss,ssss,sssss]-u[tt,s,ss,sss,ssss,sssss] for tt=1:t) <= 0) # we sell no more than stock
    end

    @objective(m, Min, 1/N^5*sum(c*u[t,s,ss,sss, ssss, sssss]-p*v[t,s,ss,sss, ssss, sssss] for t = 1:T, s = 1:N, ss = 1:N, sss = 1:N, ssss = 1:N, sssss = 1:N))


    solve(m)
    @show(getobjectivevalue(m))
end

getobjectivevalue(m) = -4.999999999987299
getobjectivevalue(m) = -9.746359999954102
getobjectivevalue(m) = -14.056380000077377
getobjectivevalue(m) = -17.541959999906425
getobjectivevalue(m) = -19.566900000038526
getobjectivevalue(m) = -19.56689999999077
getobjectivevalue(m) = -17.54195999997461
getobjectivevalue(m) = -14.056379999980393
getobjectivevalue(m) = -9.74635999998798
getobjectivevalue(m) = -4.999999999983437


Policy results:
U = 1  :  objectivevalue(m) = -4.999999999987299 <br />
U = 2  :  objectivevalue(m) = -9.746359999954102 <br />
U = 3  :  objectivevalue(m) = -14.056380000077377 <br />
U = 4  :  objectivevalue(m) = -17.541959999906425 <br />
U = 5  :  objectivevalue(m) = -19.566900000038526 <br />
U = 6  :  objectivevalue(m) = -19.56689999999077 <br />
U = 7  :  objectivevalue(m) = -17.54195999997461 <br />
U = 8  :  objectivevalue(m) = -14.056379999980393 <br />
U = 9  :  objectivevalue(m) = -9.74635999998798 <br />
U = 10 :  objectivevalue(m) = -4.999999999983437 <br />

Therefore we deduct that the optimal control using this technique is U= 5 ( the mean demand)   <br />

### Question 4.4 (Optional)

Evaluate an open-loop feedback approach on the problem.

The demand, which is uniform between 1 and 10 has a mean value of 5.5 . <br />
Therefore, we want to solve the determinstic problem where demands have been swaped for their mean value. <br />
u1 = 7   || v1 = 5.5 <br />
u2 = 7   || v2 = 5.5 <br />
u3 = 7   || v3 = 5.5 <br />
u4 = 5.5 || v4 = 5.5 <br />
u5 = 1   || v5 = 5.5 <br />
Payoff = -27.5


### Question 4.5 (optional)

Give a lower bound obtained from a two-stage approximation. 

In [ ]:
T = 5

d = zeros(T,N,N,N, N, N)
for s = 1:N, ss = 1:N, sss = 1:N, ssss = 1:N, sssss = 1:N
    d[1,s,ss,sss,ssss,sssss] = s
    d[2,s,ss,sss,ssss,sssss] = ss
    d[3,s,ss,sss,ssss,sssss] = sss
    d[4,s,ss,sss,ssss,sssss] = ssss
    d[5,s,ss,sss,ssss,sssss] = sssss
end


m = Model(solver = ClpSolver())

@variable(m,0<= u[1:T,1:N,1:N,1:N,1:N,1:N] <= M )
@variable(m, v[1:T,1:N,1:N,1:N,1:N,1:N] >= 0)

for t = 1:T, s = 1:N, ss = 1:N, sss = 1:N, ssss = 1:N, sssss = 1:N
    @constraint(m, v[t,s,ss,sss,ssss,sssss] <= d[t,s,ss,sss,ssss,sssss]) # we sell no more than demand
    @constraint(m, sum(v[tt,s,ss,sss,ssss,sssss]-u[tt,s,ss,sss,ssss,sssss] for tt=1:t) <= 0) # we sell no more than stock
end

for s = 1:N, ss = 1:N, sss = 1:N, ssss = 1:N, sssss = 1:N
    @constraint(m,u[1,s,ss,sss,ssss,sssss] == 1/N^5 * sum( u[1,t,tt,ttt,tttt,ttttt] for t=1:N, tt=1:N, ttt=1:N, tttt=1:N, ttttt=1:N))
end

@objective(m, Min, 1/N^5*sum(c*u[t,s,ss,sss, ssss, sssss]-p*v[t,s,ss,sss, ssss, sssss] for t = 1:T, s = 1:N, ss = 1:N, sss = 1:N, ssss = 1:N, sssss = 1:N))

       
solve(m)
@show(getobjectivevalue(m))

In [ ]:
Kernel Crashes

### Question 4.6 (optional)

Evaluate the repeated two-stage approach.

## Question 5

Now assume that $T=5$. 

### Question 5.1
Show that the optimal stock at the end of day is integer and no more than $N$.

Why can't the optimal stock at the end of a day superior to $N$? Because doing so yield absolutely no advantage. In fact, the maximum demand over a day is N . If the our stock is more than N, then we are sure to have stck at the end of the next day. Intuitively, the greater the stock, the lesser the probability to sell it all. And N happens to be the smallest stock, from day-to-day, that ensures the demand is satisfied no matter what. Therefore, there is no advantage to having a stock $>N$. The optimal will be no more than $N$. Again, this results holds true only because c in constant and there is no incentive to buy at a certain point time.

Now, on to the fact this optimal value will be an integer. Again, having a non integer stock yields no advantage so it will never be an optimal solution. 
The demand is an integer, but it's mean, here, is 5.5. Given the distribution is constant over $1..10$. 
Since the demand is integer, the only source of non integrity is the buying of the flowers.
I have a hard time formalizing the thing on paper but the intuition is that the idea of a non-integer optimal stock stem from the mean demand $5.5$. The idea is that everytime it is possible to sell an non-integer number of flowers, then we can also sell an integer number of flowers that is the integer part of the initial value, rounded to the maximmum. This means, that whenever it is possible to sell a non integer amount,  it is possible to sell an integer amount that will ryse the payoff. Therefore, there is no 

### Question 5.2 
How many operations are required to solve the problem by Dynamic Programming ?

First off we need to check if the problem can be resolved using dynamic programming. 
Here, we have T=5 timesteps, N=10 possible states and the incertitude is in {1,2,3,4,5,6,7,8,9,10}.

We can start by checking that the problem can indeed be solved using dynamic programming. In our case, the noise is represented by the demand each day, which is supposed to be uniform between 1 and 10. Therefore de noises are independant. 
Then, lets look at the complexity of the dynamic programming solution. As mentioned ealier, solving the dynamic programming problem means filling a $T*N$ matrix (V)  from bottom to top.  Here, N refers to the number of states possible ( i.d the stock), here it is 10. 

First, the row $(5,:) is initialized to 0 as we don't value the stock at the end of the 5th day. (N operations) 
Then, we will fill each row, from top to bottom ( (T-1 x N )operations).
In order to fill each element (t,x) of the matrix, we need to solve the minimization problem : 
$min_{u\inB(x) E[V(t+1,max( x + u - w, 0)- p * min(x + u, w) + c * u]$. 
Here, u is minimised over $B(x)$ which belongs to $[0..10]$ and since we look at a min value over an expectation, we need to additionally loop 9 times because the demand is an integer between 1 and 10. 

Overall this means that filing the matrix requires N + (T-1)*N*11*10 basic operations. Once this is done, we will have the result of the value function by dynamic programmimg. 

$10 + (5-1)*10*11*10 = 4410$ Overall, solving the rpoblem via dynamic programming requires 4410 basic operations.

### Question 5.3
Find the optimal value of the flower-girl problem by Dynamic Programming. 


In [2]:
function B(stock)
    B=0:(10-stock)
    return (B)
end

B (generic function with 1 method)

In [38]:
function DP(T)
    ## Initialization
    c = 1
    p = 2
    N = 10
    M = 7
    #Tf=10
    #Nx=9
    #Ti=1
    ## Initialization of the matrix used to store Bellman values
    V=ones(T,N+1) * 10000000000
    U=zeros(T, N+1)
    ## Compute Bellman function at final time
    for i in 1:N+1
        V[T,i]=0
    end
    ## make a backward loop on time
    #for t in T-1:-1:1
    for t in (T-1):-1:1
        ## make a loop on possible states
        for x in 0:10
            cost_x = 10000000000
            ##loop on admissible controls to obtain the best possible expected cost sta
            for u in B(x)
                cost_x_u = 0
                ## make a loop on the random perturbation to compute the expected // co
                for w in 1:10
                    ## for a given perturbation compute the cost associated to
                    ## control u and state x using the instantaneous cost and the ## Bellman function at time t+1
                    ##cost_x_u_w =(proba)*(le cout que je vais avoir depuis cet endroit + le cout pour y aller)
                    cost_x_u_w = 0.1*(V[t+1, (max( x + u - w, 0))+1] - p * min(x + u, w) + c * u)
                    ## update cost_x_u with cost_x_u_w
                    cost_x_u = cost_x_u + cost_x_u_w
                end
                ## compare the current cost_x_u to cost_x and
                ## update cost_x if cost_x_u is better than the current cost_x
                if cost_x_u <= cost_x
                    cost_x = cost_x_u
                    U[t, x+1] = u

                end
            end
        ## store cost_x in V(t,x)
        V[t,x+1] = cost_x
        end
    end
    println(V)
end

DP (generic function with 1 method)

In [39]:
DP(5)

[-19.32 -20.32 -21.32 -22.32 -23.32 -24.32 -25.32 -26.32 -27.32 -28.32 -29.32; -13.82 -14.82 -15.82 -16.82 -17.82 -18.82 -19.82 -20.82 -21.82 -22.82 -23.82; -8.32 -9.32 -10.32 -11.32 -12.32 -13.32 -14.32 -15.32 -16.32 -17.32 -18.3; -3.0 -4.0 -5.0 -6.0 -7.0 -8.0 -9.0 -9.8 -10.4 -10.8 -11.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]


The optimlal value of the problem given by the dynamic programming is $-19.32$ The good news is this validates the upper and lower bounds previously computed.

Now, lets dramaticaly increase the number of timesteps T=30

In [40]:
DP(30)

[-156.82 -157.82 -158.82 -159.82 -160.82 -161.82 -162.82 -163.82 -164.82 -165.82 -166.82; -151.32 -152.32 -153.32 -154.32 -155.32 -156.32 -157.32 -158.32 -159.32 -160.32 -161.32; -145.82 -146.82 -147.82 -148.82 -149.82 -150.82 -151.82 -152.82 -153.82 -154.82 -155.82; -140.32 -141.32 -142.32 -143.32 -144.32 -145.32 -146.32 -147.32 -148.32 -149.32 -150.32; -134.82 -135.82 -136.82 -137.82 -138.82 -139.82 -140.82 -141.82 -142.82 -143.82 -144.82; -129.32 -130.32 -131.32 -132.32 -133.32 -134.32 -135.32 -136.32 -137.32 -138.32 -139.32; -123.82 -124.82 -125.82 -126.82 -127.82 -128.82 -129.82 -130.82 -131.82 -132.82 -133.82; -118.32 -119.32 -120.32 -121.32 -122.32 -123.32 -124.32 -125.32 -126.32 -127.32 -128.32; -112.82 -113.82 -114.82 -115.82 -116.82 -117.82 -118.82 -119.82 -120.82 -121.82 -122.82; -107.32 -108.32 -109.32 -110.32 -111.32 -112.32 -113.32 -114.32 -115.32 -116.32 -117.32; -101.82 -102.82 -103.82 -104.82 -105.82 -106.82 -107.82 -108.82 -109.82 -110.82 -111.82; -96.32 -97.32 -98.32

Daily income ~5.2

In [41]:
DP(60)

[-321.82 -322.82 -323.82 -324.82 -325.82 -326.82 -327.82 -328.82 -329.82 -330.82 -331.82; -316.32 -317.32 -318.32 -319.32 -320.32 -321.32 -322.32 -323.32 -324.32 -325.32 -326.32; -310.82 -311.82 -312.82 -313.82 -314.82 -315.82 -316.82 -317.82 -318.82 -319.82 -320.82; -305.32 -306.32 -307.32 -308.32 -309.32 -310.32 -311.32 -312.32 -313.32 -314.32 -315.32; -299.82 -300.82 -301.82 -302.82 -303.82 -304.82 -305.82 -306.82 -307.82 -308.82 -309.82; -294.32 -295.32 -296.32 -297.32 -298.32 -299.32 -300.32 -301.32 -302.32 -303.32 -304.32; -288.82 -289.82 -290.82 -291.82 -292.82 -293.82 -294.82 -295.82 -296.82 -297.82 -298.82; -283.32 -284.32 -285.32 -286.32 -287.32 -288.32 -289.32 -290.32 -291.32 -292.32 -293.32; -277.82 -278.82 -279.82 -280.82 -281.82 -282.82 -283.82 -284.82 -285.82 -286.82 -287.82; -272.32 -273.32 -274.32 -275.32 -276.32 -277.32 -278.32 -279.32 -280.32 -281.32 -282.32; -266.82 -267.82 -268.82 -269.82 -270.82 -271.82 -272.82 -273.82 -274.82 -275.82 -276.82; -261.32 -262.32 -263

Daily income ~5.2

In [42]:
DP(150)

[-816.82 -817.82 -818.82 -819.82 -820.82 -821.82 -822.82 -823.82 -824.82 -825.82 -826.82; -811.32 -812.32 -813.32 -814.32 -815.32 -816.32 -817.32 -818.32 -819.32 -820.32 -821.32; -805.82 -806.82 -807.82 -808.82 -809.82 -810.82 -811.82 -812.82 -813.82 -814.82 -815.82; -800.32 -801.32 -802.32 -803.32 -804.32 -805.32 -806.32 -807.32 -808.32 -809.32 -810.32; -794.82 -795.82 -796.82 -797.82 -798.82 -799.82 -800.82 -801.82 -802.82 -803.82 -804.82; -789.32 -790.32 -791.32 -792.32 -793.32 -794.32 -795.32 -796.32 -797.32 -798.32 -799.32; -783.82 -784.82 -785.82 -786.82 -787.82 -788.82 -789.82 -790.82 -791.82 -792.82 -793.82; -778.32 -779.32 -780.32 -781.32 -782.32 -783.32 -784.32 -785.32 -786.32 -787.32 -788.32; -772.82 -773.82 -774.82 -775.82 -776.82 -777.82 -778.82 -779.82 -780.82 -781.82 -782.82; -767.32 -768.32 -769.32 -770.32 -771.32 -772.32 -773.32 -774.32 -775.32 -776.32 -777.32; -761.82 -762.82 -763.82 -764.82 -765.82 -766.82 -767.82 -768.82 -769.82 -770.82 -771.82; -756.32 -757.32 -758

.32 -185.32 -186.32 -187.32 -188.32 -189.32 -190.32 -191.32 -192.32 -193.32 -194.32; -178.82 -179.82 -180.82 -181.82 -182.82 -183.82 -184.82 -185.82 -186.82 -187.82 -188.82; -173.32 -174.32 -175.32 -176.32 -177.32 -178.32 -179.32 -180.32 -181.32 -182.32 -183.32; -167.82 -168.82 -169.82 -170.82 -171.82 -172.82 -173.82 -174.82 -175.82 -176.82 -177.82; -162.32 -163.32 -164.32 -165.32 -166.32 -167.32 -168.32 -169.32 -170.32 -171.32 -172.32; -156.82 -157.82 -158.82 -159.82 -160.82 -161.82 -162.82 -163.82 -164.82 -165.82 -166.82; -151.32 -152.32 -153.32 -154.32 -155.32 -156.32 -157.32 -158.32 -159.32 -160.32 -161.32; -145.82 -146.82 -147.82 -148.82 -149.82 -150.82 -151.82 -152.82 -153.82 -154.82 -155.82; -140.32 -141.32 -142.32 -143.32 -144.32 -145.32 -146.32 -147.32 -148.32 -149.32 -150.32; -134.82 -135.82 -136.82 -137.82 -138.82 -139.82 -140.82 -141.82 -142.82 -143.82 -144.82; -129.32 -130.32 -131.32 -132.32 -133.32 -134.32 -135.32 -136.32 -137.32 -138.32 -139.32; -123.82 -124.82 -125.82 -

Daily income ~5.44

In [43]:
DP(1000)

[-5491.82 -5492.82 -5493.82 -5494.82 -5495.82 -5496.82 -5497.82 -5498.82 -5499.82 -5500.82 -5501.82; -5486.32 -5487.32 -5488.32 -5489.32 -5490.32 -5491.32 -5492.32 -5493.32 -5494.32 -5495.32 -5496.32; -5480.82 -5481.82 -5482.82 -5483.82 -5484.82 -5485.82 -5486.82 -5487.82 -5488.82 -5489.82 -5490.82; -5475.32 -5476.32 -5477.32 -5478.32 -5479.32 -5480.32 -5481.32 -5482.32 -5483.32 -5484.32 -5485.32; -5469.82 -5470.82 -5471.82 -5472.82 -5473.82 -5474.82 -5475.82 -5476.82 -5477.82 -5478.82 -5479.82; -5464.32 -5465.32 -5466.32 -5467.32 -5468.32 -5469.32 -5470.32 -5471.32 -5472.32 -5473.32 -5474.32; -5458.82 -5459.82 -5460.82 -5461.82 -5462.82 -5463.82 -5464.82 -5465.82 -5466.82 -5467.82 -5468.82; -5453.32 -5454.32 -5455.32 -5456.32 -5457.32 -5458.32 -5459.32 -5460.32 -5461.32 -5462.32 -5463.32; -5447.82 -5448.82 -5449.82 -5450.82 -5451.82 -5452.82 -5453.82 -5454.82 -5455.82 -5456.82 -5457.82; -5442.32 -5443.32 -5444.32 -5445.32 -5446.32 -5447.32 -5448.32 -5449.32 -5450.32 -5451.32 -5452.32;

82 -4935.82 -4936.82 -4937.82 -4938.82 -4939.82 -4940.82; -4925.32 -4926.32 -4927.32 -4928.32 -4929.32 -4930.32 -4931.32 -4932.32 -4933.32 -4934.32 -4935.32; -4919.82 -4920.82 -4921.82 -4922.82 -4923.82 -4924.82 -4925.82 -4926.82 -4927.82 -4928.82 -4929.82; -4914.32 -4915.32 -4916.32 -4917.32 -4918.32 -4919.32 -4920.32 -4921.32 -4922.32 -4923.32 -4924.32; -4908.82 -4909.82 -4910.82 -4911.82 -4912.82 -4913.82 -4914.82 -4915.82 -4916.82 -4917.82 -4918.82; -4903.32 -4904.32 -4905.32 -4906.32 -4907.32 -4908.32 -4909.32 -4910.32 -4911.32 -4912.32 -4913.32; -4897.82 -4898.82 -4899.82 -4900.82 -4901.82 -4902.82 -4903.82 -4904.82 -4905.82 -4906.82 -4907.82; -4892.32 -4893.32 -4894.32 -4895.32 -4896.32 -4897.32 -4898.32 -4899.32 -4900.32 -4901.32 -4902.32; -4886.82 -4887.82 -4888.82 -4889.82 -4890.82 -4891.82 -4892.82 -4893.82 -4894.82 -4895.82 -4896.82; -4881.32 -4882.32 -4883.32 -4884.32 -4885.32 -4886.32 -4887.32 -4888.32 -4889.32 -4890.32 -4891.32; -4875.82 -4876.82 -4877.82 -4878.82 -4879.

32 -4223.32 -4224.32 -4225.32 -4226.32 -4227.32 -4228.32 -4229.32 -4230.32 -4231.32; -4215.82 -4216.82 -4217.82 -4218.82 -4219.82 -4220.82 -4221.82 -4222.82 -4223.82 -4224.82 -4225.82; -4210.32 -4211.32 -4212.32 -4213.32 -4214.32 -4215.32 -4216.32 -4217.32 -4218.32 -4219.32 -4220.32; -4204.82 -4205.82 -4206.82 -4207.82 -4208.82 -4209.82 -4210.82 -4211.82 -4212.82 -4213.82 -4214.82; -4199.32 -4200.32 -4201.32 -4202.32 -4203.32 -4204.32 -4205.32 -4206.32 -4207.32 -4208.32 -4209.32; -4193.82 -4194.82 -4195.82 -4196.82 -4197.82 -4198.82 -4199.82 -4200.82 -4201.82 -4202.82 -4203.82; -4188.32 -4189.32 -4190.32 -4191.32 -4192.32 -4193.32 -4194.32 -4195.32 -4196.32 -4197.32 -4198.32; -4182.82 -4183.82 -4184.82 -4185.82 -4186.82 -4187.82 -4188.82 -4189.82 -4190.82 -4191.82 -4192.82; -4177.32 -4178.32 -4179.32 -4180.32 -4181.32 -4182.32 -4183.32 -4184.32 -4185.32 -4186.32 -4187.32; -4171.82 -4172.82 -4173.82 -4174.82 -4175.82 -4176.82 -4177.82 -4178.82 -4179.82 -4180.82 -4181.82; -4166.32 -4167.

.32 -3667.32 -3668.32 -3669.32 -3670.32; -3654.82 -3655.82 -3656.82 -3657.82 -3658.82 -3659.82 -3660.82 -3661.82 -3662.82 -3663.82 -3664.82; -3649.32 -3650.32 -3651.32 -3652.32 -3653.32 -3654.32 -3655.32 -3656.32 -3657.32 -3658.32 -3659.32; -3643.82 -3644.82 -3645.82 -3646.82 -3647.82 -3648.82 -3649.82 -3650.82 -3651.82 -3652.82 -3653.82; -3638.32 -3639.32 -3640.32 -3641.32 -3642.32 -3643.32 -3644.32 -3645.32 -3646.32 -3647.32 -3648.32; -3632.82 -3633.82 -3634.82 -3635.82 -3636.82 -3637.82 -3638.82 -3639.82 -3640.82 -3641.82 -3642.82; -3627.32 -3628.32 -3629.32 -3630.32 -3631.32 -3632.32 -3633.32 -3634.32 -3635.32 -3636.32 -3637.32; -3621.82 -3622.82 -3623.82 -3624.82 -3625.82 -3626.82 -3627.82 -3628.82 -3629.82 -3630.82 -3631.82; -3616.32 -3617.32 -3618.32 -3619.32 -3620.32 -3621.32 -3622.32 -3623.32 -3624.32 -3625.32 -3626.32; -3610.82 -3611.82 -3612.82 -3613.82 -3614.82 -3615.82 -3616.82 -3617.82 -3618.82 -3619.82 -3620.82; -3605.32 -3606.32 -3607.32 -3608.32 -3609.32 -3610.32 -3611

3093.82 -3094.82 -3095.82 -3096.82 -3097.82 -3098.82 -3099.82 -3100.82 -3101.82 -3102.82 -3103.82; -3088.32 -3089.32 -3090.32 -3091.32 -3092.32 -3093.32 -3094.32 -3095.32 -3096.32 -3097.32 -3098.32; -3082.82 -3083.82 -3084.82 -3085.82 -3086.82 -3087.82 -3088.82 -3089.82 -3090.82 -3091.82 -3092.82; -3077.32 -3078.32 -3079.32 -3080.32 -3081.32 -3082.32 -3083.32 -3084.32 -3085.32 -3086.32 -3087.32; -3071.82 -3072.82 -3073.82 -3074.82 -3075.82 -3076.82 -3077.82 -3078.82 -3079.82 -3080.82 -3081.82; -3066.32 -3067.32 -3068.32 -3069.32 -3070.32 -3071.32 -3072.32 -3073.32 -3074.32 -3075.32 -3076.32; -3060.82 -3061.82 -3062.82 -3063.82 -3064.82 -3065.82 -3066.82 -3067.82 -3068.82 -3069.82 -3070.82; -3055.32 -3056.32 -3057.32 -3058.32 -3059.32 -3060.32 -3061.32 -3062.32 -3063.32 -3064.32 -3065.32; -3049.82 -3050.82 -3051.82 -3052.82 -3053.82 -3054.82 -3055.82 -3056.82 -3057.82 -3058.82 -3059.82; -3044.32 -3045.32 -3046.32 -3047.32 -3048.32 -3049.32 -3050.32 -3051.32 -3052.32 -3053.32 -3054.32; -

82 -2104.82 -2105.82 -2106.82 -2107.82 -2108.82 -2109.82 -2110.82 -2111.82 -2112.82 -2113.82; -2098.32 -2099.32 -2100.32 -2101.32 -2102.32 -2103.32 -2104.32 -2105.32 -2106.32 -2107.32 -2108.32; -2092.82 -2093.82 -2094.82 -2095.82 -2096.82 -2097.82 -2098.82 -2099.82 -2100.82 -2101.82 -2102.82; -2087.32 -2088.32 -2089.32 -2090.32 -2091.32 -2092.32 -2093.32 -2094.32 -2095.32 -2096.32 -2097.32; -2081.82 -2082.82 -2083.82 -2084.82 -2085.82 -2086.82 -2087.82 -2088.82 -2089.82 -2090.82 -2091.82; -2076.32 -2077.32 -2078.32 -2079.32 -2080.32 -2081.32 -2082.32 -2083.32 -2084.32 -2085.32 -2086.32; -2070.82 -2071.82 -2072.82 -2073.82 -2074.82 -2075.82 -2076.82 -2077.82 -2078.82 -2079.82 -2080.82; -2065.32 -2066.32 -2067.32 -2068.32 -2069.32 -2070.32 -2071.32 -2072.32 -2073.32 -2074.32 -2075.32; -2059.82 -2060.82 -2061.82 -2062.82 -2063.82 -2064.82 -2065.82 -2066.82 -2067.82 -2068.82 -2069.82; -2054.32 -2055.32 -2056.32 -2057.32 -2058.32 -2059.32 -2060.32 -2061.32 -2062.32 -2063.32 -2064.32; -2048.

.82 -1548.82 -1549.82 -1550.82 -1551.82 -1552.82; -1537.32 -1538.32 -1539.32 -1540.32 -1541.32 -1542.32 -1543.32 -1544.32 -1545.32 -1546.32 -1547.32; -1531.82 -1532.82 -1533.82 -1534.82 -1535.82 -1536.82 -1537.82 -1538.82 -1539.82 -1540.82 -1541.82; -1526.32 -1527.32 -1528.32 -1529.32 -1530.32 -1531.32 -1532.32 -1533.32 -1534.32 -1535.32 -1536.32; -1520.82 -1521.82 -1522.82 -1523.82 -1524.82 -1525.82 -1526.82 -1527.82 -1528.82 -1529.82 -1530.82; -1515.32 -1516.32 -1517.32 -1518.32 -1519.32 -1520.32 -1521.32 -1522.32 -1523.32 -1524.32 -1525.32; -1509.82 -1510.82 -1511.82 -1512.82 -1513.82 -1514.82 -1515.82 -1516.82 -1517.82 -1518.82 -1519.82; -1504.32 -1505.32 -1506.32 -1507.32 -1508.32 -1509.32 -1510.32 -1511.32 -1512.32 -1513.32 -1514.32; -1498.82 -1499.82 -1500.82 -1501.82 -1502.82 -1503.82 -1504.82 -1505.82 -1506.82 -1507.82 -1508.82; -1493.32 -1494.32 -1495.32 -1496.32 -1497.32 -1498.32 -1499.32 -1500.32 -1501.32 -1502.32 -1503.32; -1487.82 -1488.82 -1489.82 -1490.82 -1491.82 -1492

 -979.32 -980.32 -981.32 -982.32 -983.32 -984.32 -985.32 -986.32; -970.82 -971.82 -972.82 -973.82 -974.82 -975.82 -976.82 -977.82 -978.82 -979.82 -980.82; -965.32 -966.32 -967.32 -968.32 -969.32 -970.32 -971.32 -972.32 -973.32 -974.32 -975.32; -959.82 -960.82 -961.82 -962.82 -963.82 -964.82 -965.82 -966.82 -967.82 -968.82 -969.82; -954.32 -955.32 -956.32 -957.32 -958.32 -959.32 -960.32 -961.32 -962.32 -963.32 -964.32; -948.82 -949.82 -950.82 -951.82 -952.82 -953.82 -954.82 -955.82 -956.82 -957.82 -958.82; -943.32 -944.32 -945.32 -946.32 -947.32 -948.32 -949.32 -950.32 -951.32 -952.32 -953.32; -937.82 -938.82 -939.82 -940.82 -941.82 -942.82 -943.82 -944.82 -945.82 -946.82 -947.82; -932.32 -933.32 -934.32 -935.32 -936.32 -937.32 -938.32 -939.32 -940.32 -941.32 -942.32; -926.82 -927.82 -928.82 -929.82 -930.82 -931.82 -932.82 -933.82 -934.82 -935.82 -936.82; -921.32 -922.32 -923.32 -924.32 -925.32 -926.32 -927.32 -928.32 -929.32 -930.32 -931.32; -915.82 -916.82 -917.82 -918.82 -919.82 -920

Daily income ~5.5
Overall, as the number on timesteps increase, the average daily gain converges toward 5.5, which totally makes sense as it is the average demand in flowers ( and $p=2$ and $c=1$)